In [ ]:
import time

In [1]:
MODEL_NAME = "premier_league"
MODEL_VERSION = "v1.0"
MODEL_TYPE = "win_outcome"

In [4]:
# UPDATING THE GITHUB OF THE DATASETS
!git add -A
!git commit -m f"updates: {MODEL_NAME}-{MODEL_VERSION}-{MODEL_TYPE}: bot commit - {time.time()}"
!git push

[main 6a692f8] updates: {MODEL_NAME}-{MODEL_VERSION}-{MODEL_TYPE}: bot commit - {time.time()}
 1 file changed, 22 insertions(+), 13 deletions(-)


Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 2 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 771 bytes | 771.00 KiB/s, done.
Total 4 (delta 3), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To https://github.com/lebyanelm/neural-trained-models.git
   501bf33..6a692f8  main -> main
